In [1]:
import requests
import json

import pandas as pd

In [2]:
with open('github.token', 'r') as f:
    token = f.read()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


In [26]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issues(first:100, states:OPEN) {
    edges{
    node {
          number
          title
          url
          projectCards(first:10) {
              edges {
                  node {
                      column {name}
                      project {name}
                  }
              }
          }
          reactionGroups {
            content
            users(first:50) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }   
            }    
    }
    }
    }
  }
}
"""

result = run_query(query)  # Execute the query

result

{'data': {'repository': {'issues': {'edges': [{'node': {'number': 4,
       'title': 'Hut23/Skills development',
       'url': 'https://github.com/alan-turing-institute/Hut23/issues/4',
       'projectCards': {'edges': [{'node': {'column': {'name': 'Done'},
           'project': {'name': 'Planning'}}}]},
       'reactionGroups': [{'content': 'THUMBS_UP', 'users': {'edges': []}},
        {'content': 'THUMBS_DOWN', 'users': {'edges': []}},
        {'content': 'LAUGH', 'users': {'edges': []}},
        {'content': 'HOORAY', 'users': {'edges': []}},
        {'content': 'CONFUSED', 'users': {'edges': []}},
        {'content': 'HEART', 'users': {'edges': []}},
        {'content': 'ROCKET', 'users': {'edges': []}},
        {'content': 'EYES', 'users': {'edges': []}}]}},
     {'node': {'number': 9,
       'title': 'LRF/001/c Data Diff',
       'url': 'https://github.com/alan-turing-institute/Hut23/issues/9',
       'projectCards': {'edges': []},
       'reactionGroups': [{'content': 'THUMBS_UP'

In [34]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    project(number:2) {
        name
        body
        columns(first:20) {
            edges {
                node {
                    name
                    cards (first:100) {
                        edges {
                            node {
                                url                           
                            }
                        }
                        
                    }
                }
            }
        }
    }
  }
}
"""

result = run_query(query)  # Execute the query

result

{'data': {'repository': {'project': {'name': 'Project tracker',
    'body': 'To track prospective and active projects',
    'columns': {'edges': [{'node': {'name': 'Suggested',
        'cards': {'edges': [{'node': {'url': 'https://github.com/alan-turing-institute/Hut23/projects/2#card-22643736'}},
          {'node': {'url': 'https://github.com/alan-turing-institute/Hut23/projects/2#card-22509032'}},
          {'node': {'url': 'https://github.com/alan-turing-institute/Hut23/projects/2#card-22421099'}},
          {'node': {'url': 'https://github.com/alan-turing-institute/Hut23/projects/2#card-14896425'}},
          {'node': {'url': 'https://github.com/alan-turing-institute/Hut23/projects/2#card-19507533'}},
          {'node': {'url': 'https://github.com/alan-turing-institute/Hut23/projects/2#card-19818861'}},
          {'node': {'url': 'https://github.com/alan-turing-institute/Hut23/projects/2#card-19507655'}},
          {'node': {'url': 'https://github.com/alan-turing-institute/Hut23/pr

In [4]:
issue = result['data']['repository']['issue']
reactions = issue['reactionGroups']
reactions

[{'content': 'THUMBS_UP',
  'users': {'edges': [{'node': {'login': 'tomaslaz',
      'name': 'Tomas Lazauskas'}},
    {'node': {'login': 'thobson88', 'name': 'Tim Hobson'}},
    {'node': {'login': 'nbarlowATI', 'name': None}},
    {'node': {'login': 'JimMadge', 'name': 'Jim Madge'}},
    {'node': {'login': 'sgibson91', 'name': 'Sarah Gibson'}}]}},
 {'content': 'THUMBS_DOWN',
  'users': {'edges': [{'node': {'login': 'Giovanni1085',
      'name': 'Giovanni Colavizza'}},
    {'node': {'login': 'myyong', 'name': None}}]}},
 {'content': 'LAUGH',
  'users': {'edges': [{'node': {'login': 'jemrobinson',
      'name': 'James Robinson'}},
    {'node': {'login': 'evelinag', 'name': 'Evelina Gabasova'}},
    {'node': {'login': 'LouiseABowler', 'name': 'Louise Bowler'}},
    {'node': {'login': 'crangelsmith', 'name': 'Camila Rangel Smith'}},
    {'node': {'login': 'miguelmorin', 'name': None}},
    {'node': {'login': 'jack89roberts', 'name': 'Jack Roberts'}},
    {'node': {'login': 'OscartGiles', '

In [5]:
emojis = {
    'CONFUSED': '😕',
    'EYES': '👀',
    'HEART': '❤️',
    'HOORAY': '🎉',
    'LAUGH': '😄',
    'ROCKET': '🚀',
    'THUMBS_DOWN': '👎',
    'THUMBS_UP': '👍'
}


In [8]:
df = dict()
for reaction in reactions:
    content = reaction['content']
    users = [x['node']['login'] for x in reaction['users']['edges']]
    df[content] = users
             
df = pd.Series(df)
df.index = [emojis[idx] for idx in df.index]
df.name = issue['title']
df

👍     [tomaslaz, thobson88, nbarlowATI, JimMadge, sg...
👎                                [Giovanni1085, myyong]
😄     [jemrobinson, evelinag, LouiseABowler, crangel...
🎉                                                    []
😕                                                    []
❤️                                                   []
🚀                                                    []
👀                                                    []
Name: Clean air project (Theo Damoulas), dtype: object